In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/processed_data.csv")
df_encuestas = pd.read_csv("../data/encuestas.csv")
# === Procesamiento de fechas ===
df["createdAt"] = pd.to_datetime(df["createdAt"])
df_encuestas["date"] = pd.to_datetime(df_encuestas["date"])
df["date"] = df["createdAt"].dt.floor("D")
df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)
df_encuestas["date"] = pd.to_datetime(df_encuestas["date"]).dt.tz_localize(None)

/tmp/ipykernel_4654/3492966212.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed_data.csv")


In [3]:
# === Agregación diaria ===
import numpy as np
df["date"] = pd.to_datetime(df["date"])

def iqr(x):
    return np.percentile(x, 75) - np.percentile(x, 25)

df_daily = df.set_index('date').groupby('date').agg({
    "score_positive": ["mean", "median", "std", iqr],
    "score_negative": ["mean", "median", "std", iqr],
    "score_neutral": ["mean", "median", "std", iqr],
    "retweetCount": "mean",
    "replyCount": "mean",
    "likeCount": "mean",
    "quoteCount": "mean",
    **{f"robertuito_{i}": "mean" for i in range(768)}
})
df_daily.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_daily.columns]
df_daily = df_daily.reset_index()

# === Asignar valores CADEM desde domingo hasta sábado ===

start_date = df_encuestas["date"].min()
end_date = df_encuestas["date"].max() + pd.Timedelta(days=6)
df_cadem_expandido = pd.DataFrame({"date": pd.date_range(start=start_date, end=end_date, freq="D")})
df_cadem_expandido["semana_objetivo"] = df_cadem_expandido["date"].apply(
    lambda d: d + pd.to_timedelta(6 - d.weekday(), unit="D")
)
df_encuestas_ren = df_encuestas.rename(columns={"date": "semana_objetivo"})
df_cadem_expandido = df_cadem_expandido.merge(df_encuestas_ren, on="semana_objetivo", how="left")
df_daily = df_daily.merge(df_cadem_expandido[["date", "aprobacion_boric", "desaprobacion_boric"]], on="date", how="left")

selected_embeddings = [
    'robertuito_7_mean', 'robertuito_214_mean', 'robertuito_231_mean', 'robertuito_244_mean',
    'robertuito_298_mean', 'robertuito_311_mean', 'robertuito_328_mean', 'robertuito_373_mean',
    'robertuito_551_mean', 'robertuito_625_mean', 'robertuito_650_mean', 'aprobacion_boric', 'desaprobacion_boric'
]

columns_to_keep = ['date'] + \
    [col for col in df_daily.columns if col.startswith('score_')] + \
    ['retweetCount_mean', 'replyCount_mean', 'likeCount_mean', 'quoteCount_mean'] + \
    selected_embeddings

df_daily = df_daily[columns_to_keep]

df_daily.to_csv("df_daily.csv", index=False)
# === Resultado final ===
df_daily.tail(20)

,date,score_positive_mean,score_positive_median,score_positive_std,score_positive_iqr,score_negative_mean,score_negative_median,score_negative_std,score_negative_iqr,score_neutral_mean,...,robertuito_244_mean,robertuito_298_mean,robertuito_311_mean,robertuito_328_mean,robertuito_373_mean,robertuito_551_mean,robertuito_625_mean,robertuito_650_mean,aprobacion_boric,desaprobacion_boric
1135,2025-04-09,0.217047,0.135862,0.205628,0.276491,0.344908,0.290622,0.271758,0.520850,0.438045,...,0.156146,-0.142408,-0.312941,-0.443205,-0.286317,0.542216,0.307077,0.691746,0.27,0.69
1136,2025-04-10,0.138421,0.098182,0.120970,0.122070,0.322291,0.195645,0.272770,0.346265,0.539288,...,0.216831,-0.156049,-0.339438,-0.368781,-0.352118,0.768716,0.216855,0.697980,0.27,0.69
1137,2025-04-11,0.124856,0.052631,0.170930,0.174034,0.555379,0.615855,0.344979,0.673928,0.319765,...,0.268793,-0.155815,-0.313947,-0.277939,-0.076778,0.632417,0.217541,0.759066,0.27,0.69
1138,2025-04-12,0.142172,0.061062,0.216005,0.110704,0.547127,0.703253,0.387416,0.772465,0.310701,...,0.288078,-0.094968,-0.330739,-0.394073,-0.056413,0.653828,0.225736,0.799938,0.27,0.69
1139,2025-04-13,0.136304,0.045424,0.244017,0.086266,0.541885,0.586057,0.328079,0.590917,0.321811,...,0.159725,-0.182452,-0.376078,-0.302546,-0.313633,0.644501,0.492382,0.667191,0.27,0.69
1140,2025-04-14,0.211345,0.124630,0.217649,0.264187,0.383933,0.161681,0.349808,0.639694,0.404722,...,0.188072,-0.145545,-0.499798,-0.306816,-0.321678,0.625772,0.363233,0.711215,0.32,0.66
1141,2025-04-15,0.128366,0.050465,0.150865,0.215594,0.494115,0.523161,0.318907,0.477699,0.377519,...,0.253581,-0.195544,-0.456473,-0.330385,-0.089474,0.618533,0.287464,0.722017,0.32,0.66
1142,2025-04-16,0.045629,0.025729,0.041117,0.065799,0.675566,0.806775,0.290313,0.394920,0.278805,...,0.256611,-0.114723,-0.099788,-0.327914,-0.072077,0.410245,0.332907,0.750286,0.32,0.66
1143,2025-04-17,0.100585,0.038468,0.142500,0.102767,0.563863,0.651651,0.336267,0.677784,0.335552,...,0.263364,-0.176818,-0.167999,-0.276157,-0.060396,0.469891,0.214105,0.699416,0.32,0.66
1144,2025-04-18,0.121163,0.037022,0.198697,0.097051,0.561869,0.641484,0.345604,0.711235,0.316968,...,0.239779,-0.135137,-0.213903,-0.313363,-0.068893,0.582606,0.261639,0.652239,0.32,0.66


In [31]:
df_sample = df_daily.sample(n=9, random_state=42)
df_sample.tail(9)
#df_sample.to_csv("df_sample.csv", index=False)

,date,score_positive_mean,score_positive_median,score_positive_std,score_positive_iqr,score_negative_mean,score_negative_median,score_negative_std,score_negative_iqr,score_neutral_mean,...,robertuito_244_mean,robertuito_298_mean,robertuito_311_mean,robertuito_328_mean,robertuito_373_mean,robertuito_551_mean,robertuito_625_mean,robertuito_650_mean,aprobacion_boric,desaprobacion_boric
964,2024-10-20,0.035570,0.021439,0.031119,0.045998,0.682932,0.712711,0.233428,0.292584,0.281498,...,0.221871,-0.106467,-0.241366,-0.375420,-0.164703,0.529711,0.012874,0.668411,0.27,0.65
265,2022-11-21,0.154477,0.110277,0.175383,0.079807,0.357113,0.243915,0.282436,0.486893,0.488411,...,0.127999,-0.056653,-0.341946,-0.399479,-0.067771,0.606015,0.027495,0.577600,0.29,0.63
109,2022-06-18,0.101866,0.071759,0.098205,0.140428,0.513235,0.495652,0.326197,0.574013,0.384899,...,0.072239,-0.264764,-0.388409,-0.405294,-0.296203,0.821129,0.522879,0.658952,0.40,0.54
299,2022-12-25,0.166909,0.111940,0.175275,0.219947,0.381016,0.305485,0.292725,0.467634,0.452075,...,0.212631,-0.120228,-0.417427,-0.355834,-0.139656,0.563425,0.271370,0.689042,0.30,0.66
844,2024-06-22,0.109374,0.067957,0.165040,0.098428,0.495413,0.407295,0.311536,0.555145,0.395213,...,0.094425,-0.119221,-0.298220,-0.496715,-0.010279,0.494574,0.254358,0.488138,0.36,0.57
170,2022-08-18,0.107403,0.034761,0.138415,0.167233,0.525760,0.505585,0.355621,0.706210,0.366837,...,0.268236,-0.050862,-0.219677,-0.331000,0.101028,0.439929,0.329171,0.591723,0.37,0.56
881,2024-07-29,0.105669,0.036732,0.155947,0.095036,0.597757,0.558421,0.310209,0.432329,0.296574,...,0.183766,-0.366753,-0.217047,-0.450771,-0.125285,0.388503,0.351873,0.655148,0.35,0.57
549,2023-09-01,0.109046,0.038993,0.155406,0.154074,0.557790,0.570211,0.321569,0.645585,0.333164,...,0.242240,-0.171269,-0.265045,-0.243853,-0.073325,0.674901,0.324191,0.727082,0.29,0.65
937,2024-09-23,0.080567,0.060001,0.067580,0.075801,0.536130,0.505695,0.260566,0.425535,0.383303,...,0.250973,-0.085604,-0.223664,-0.347183,0.116975,0.673040,0.125505,0.624325,0.34,0.57


In [37]:
import pandas as pd

# === Cargar las bases ===
# 9 fechas seleccionadas
df_sample = pd.read_csv("df_sample.csv")

# Base completa de tweets procesados
df_processed = pd.read_csv("../data/processed_data.csv")  # Ajusta la ruta si es diferente

# === Extraer las fechas únicas de df_sample ===
fechas_objetivo = pd.to_datetime(df_sample["date"]).dt.date.unique()

# Asegurar que la fecha en processed también esté como date
df_processed["date"] = pd.to_datetime(df_processed["date"]).dt.date

# === Filtrar tweets correspondientes a esas fechas ===
df_tweets_sample = df_processed[df_processed["date"].isin(fechas_objetivo)]

# === (Opcional) Solo dejar columnas relevantes si quieres: scores + embeddings ===
# columnas_scores_embeddings = [c for c in df_tweets_sample.columns if "score_" in c or "robertuito_" in c]
# df_tweets_sample = df_tweets_sample[["date", "text"] + columnas_scores_embeddings]
columns = ['type', 'id', 'url', 'twitterUrl', 'source', 'lang', 'bookmarkCount', 'isReply', 'inReplyToId', 'conversationId', 'inReplyToUserId', 'inReplyToUsername', 'isPinned', 'author']
df_tweets_sample.drop(columns=columns)
# === Guardar los tweets seleccionados ===
#df_tweets_sample.to_csv("data/features/tweets_sample.csv", index=False)

/tmp/ipykernel_4831/1700344981.py:8: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_processed = pd.read_csv("../data/processed_data.csv")  # Ajusta la ruta si es diferente


,text,retweetCount,replyCount,likeCount,quoteCount,viewCount,createdAt,extendedEntities,card,place,...,robertuito_758,robertuito_759,robertuito_760,robertuito_761,robertuito_762,robertuito_763,robertuito_764,robertuito_765,robertuito_766,robertuito_767
2221,trabajadores codelco saben perfectamente fundi...,4,0,2,0,0,2022-06-18 22:22:11+00:00,{'media': [{'additional_media_info': {'monetiz...,NaN,{},...,0.058200,-0.643088,-0.325140,1.149881,0.543640,-0.532861,-0.603201,-1.435695,-0.258784,0.597884
2222,presidente gabriel boric quiero enfático traba...,1,2,4,5,0,2022-06-18 01:31:30+00:00,{'media': [{'display_url': 'pic.twitter.com/Uo...,NaN,{},...,0.194147,0.622122,-0.835796,1.475485,0.256167,0.187872,-1.554435,-0.509360,-0.280889,1.315554
2223,cnnprime presidente gabriel boric refiere cier...,401,161,1561,94,0,2022-06-18 01:24:39+00:00,{'media': [{'additional_media_info': {'monetiz...,NaN,{},...,0.843195,0.214948,-0.469726,2.077108,0.096812,-0.380900,-1.279943,-0.405662,0.315992,1.130102
2224,boric chile región campea terrorismo violencia...,81,6,104,3,0,2022-06-18 23:28:17+00:00,{},NaN,{},...,0.536430,0.572491,-0.069819,2.026211,0.095109,-0.018150,-0.815376,-1.586243,0.231185,1.160972
2225,presidente boric anuncia cierre fundición code...,0,0,1,0,0,2022-06-18 21:58:54+00:00,{},NaN,{},...,0.594781,0.535052,-0.648841,1.779601,0.041602,0.176959,-0.805120,-0.468686,-0.223482,1.091252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19325,falta consecuencia alarmante sinvergüenza enri...,212,10,407,1,12909,2024-10-20 11:02:09+00:00,{'media': [{'additional_media_info': {'monetiz...,NaN,{},...,0.079205,0.360542,0.128430,0.616831,-0.043329,0.112786,-0.065356,-1.564583,0.323355,0.853497
19326,rector udp carlos peña repasa actualidad pasad...,40,4,55,1,5574,2024-10-20 13:15:00+00:00,{'media': [{'allow_download_status': {'allow_d...,NaN,{},...,0.918296,1.135906,-0.497101,1.951752,0.419649,-0.254071,-0.505338,-1.247448,-0.023400,0.504286
19327,boric dejó monsalve cargo pesar conocer denunc...,1322,58,2481,27,118282,2024-10-20 21:45:44+00:00,{'media': [{'additional_media_info': {'monetiz...,NaN,{},...,0.354681,0.838294,-0.337531,1.287794,0.038788,-0.267108,-0.310759,-1.046254,0.197980,1.119002
19328,cadem incluye conferencia prensa boric viernes...,272,8,645,5,9751,2024-10-20 23:38:07+00:00,{'media': [{'allow_download_status': {'allow_d...,NaN,{},...,0.628705,0.296039,-0.475335,1.554445,0.854614,-0.576151,-1.360016,-1.393528,0.105001,0.305165


In [41]:
selected_embeddings = [
    'robertuito_7', 'robertuito_214', 'robertuito_231', 'robertuito_244',
    'robertuito_298', 'robertuito_311', 'robertuito_328', 'robertuito_373',
    'robertuito_551', 'robertuito_625', 'robertuito_650'
]

columns_to_keep = ['date'] + \
    ['text'] + \
    [col for col in df_tweets_sample.columns if col.startswith('score_')] + \
    ['retweetCount', 'replyCount', 'likeCount', 'quoteCount'] + \
    selected_embeddings

df_tweets_sample = df_tweets_sample[columns_to_keep]

df_tweets_sample.to_csv("tweets_sample.csv", index=False)

In [1]:
import pandas as pd

# === Cargar las bases ===
# 9 fechas seleccionadas
df_sample = pd.read_csv("df_sample.csv")

# Base completa de tweets procesados
df_processed = pd.read_csv("../respaldos/data/processed_data.csv")  # Ajusta la ruta si es diferente

# === Extraer las fechas únicas de df_sample ===
fechas_objetivo = pd.to_datetime(df_sample["date"]).dt.date.unique()

# Asegurar que la fecha en processed también esté como date
df_processed["date"] = pd.to_datetime(df_processed["createdAt"]).dt.date

# === Filtrar tweets correspondientes a esas fechas ===
df_tweets_sample = df_processed[df_processed["date"].isin(fechas_objetivo)]

# === (Opcional) Solo dejar columnas relevantes si quieres: scores + embeddings ===
# columnas_scores_embeddings = [c for c in df_tweets_sample.columns if "score_" in c or "robertuito_" in c]
# df_tweets_sample = df_tweets_sample[["date", "text"] + columnas_scores_embeddings]
columns = ['type', 'id', 'url', 'twitterUrl', 'source', 'lang', 'bookmarkCount', 'isReply', 'inReplyToId', 'conversationId', 'inReplyToUserId', 'inReplyToUsername', 'isPinned', 'author']
df_tweets_sample.drop(columns=columns)
# === Guardar los tweets seleccionados ===
#df_tweets_sample.to_csv("data/features/tweets_sample.csv", index=False)
selected_embeddings = [
    'robertuito_7', 'robertuito_214', 'robertuito_231', 'robertuito_244',
    'robertuito_298', 'robertuito_311', 'robertuito_328', 'robertuito_373',
    'robertuito_551', 'robertuito_625', 'robertuito_650'
]

columns_to_keep = ['date'] + \
    ['text'] + \
    [col for col in df_tweets_sample.columns if col.startswith('score_')] + \
    ['retweetCount', 'replyCount', 'likeCount', 'quoteCount'] + \
    selected_embeddings

df_tweets_sample = df_tweets_sample[columns_to_keep]
df_tweets_sample

,date,text,score_label,score_negative,score_neutral,score_positive,retweetCount,replyCount,likeCount,quoteCount,...,robertuito_214,robertuito_231,robertuito_244,robertuito_298,robertuito_311,robertuito_328,robertuito_373,robertuito_551,robertuito_625,robertuito_650
1372,2022-06-18,presidente gabriel boric cierre división venta...,0.656658,0.191720,0.656658,0.151622,54,7,240,3,...,-0.045112,1.336355,0.289618,-0.272051,-0.682547,-0.570402,-0.184567,0.573433,0.416186,0.848420
1373,2022-06-18,tememos presidente fachos rqliosy valiente piñ...,0.926579,0.926579,0.058476,0.014945,160,29,675,4,...,0.284107,0.898472,0.337553,0.382185,0.172362,-0.112773,0.072645,0.812215,0.235022,0.386946
1374,2022-06-18,gabriel boric reformador mundo valor reformar ...,0.534955,0.218176,0.534955,0.246869,341,10,1285,12,...,0.544273,1.139409,0.139819,-0.629338,-0.895329,-0.356579,-0.286383,0.634344,0.263681,0.796779
3456,2022-11-21,encuesta cadem aprobación presidente gabriel b...,0.564825,0.332254,0.564825,0.102921,63,69,192,2,...,0.232275,1.382148,0.060672,-0.079017,-0.659129,-1.094097,0.089225,0.993159,-0.753244,0.912900
3457,2022-11-21,presidente boric encabeza comité político bibl...,0.561914,0.144538,0.561914,0.293548,56,58,317,11,...,0.228479,1.580846,0.106794,-0.468519,-0.217278,-0.505368,-0.375507,0.782654,-0.467888,0.721455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93343,2024-10-20,it over boric,0.486621,0.325071,0.486621,0.188309,7,1,22,0,...,0.739650,-0.519003,0.277571,0.749902,-0.973399,-0.261252,0.540082,-0.289787,-0.140381,0.039554
93344,2024-10-20,boric presidente chile apoyo chile quiere,0.482653,0.131770,0.385577,0.482653,4,0,4,0,...,-0.706627,1.146385,0.093904,-0.707905,-0.726616,-0.535337,-0.516429,1.384583,0.136846,0.769701
93345,2024-10-20,columna mauricio morale roja directa,0.607331,0.283610,0.607331,0.109058,23,17,43,0,...,1.444498,1.240190,0.261230,-0.342499,-0.599346,-0.878888,-0.164175,0.108863,-0.180168,0.408316
93346,2024-10-20,alberto mayol izquierda embargo piedad gobiern...,0.578442,0.356041,0.578442,0.065518,16,4,29,1,...,-0.038826,1.276718,0.159339,-0.019277,-0.129131,-0.734194,-0.044304,0.400607,-0.090803,1.085869


In [2]:
df_tweets_sample.to_csv("tweets_sample_full.csv", index=False)

In [ ]:
df_processed = pd.read_csv("../respaldos/data/processed_data.csv")
df_processed["date"] = pd.to_datetime(df_processed["createdAt"]).dt.date
df_tweets = df_processed[df_processed["date"].isin(fechas_objetivo)]
columns = ['type', 'id', 'url', 'twitterUrl', 'source', 'lang', 'bookmarkCount', 'isReply', 'inReplyToId', 'conversationId', 'inReplyToUserId', 'inReplyToUsername', 'isPinned', 'author']
df_tweets.drop(columns=columns)
selected_embeddings = [
    'robertuito_7', 'robertuito_214', 'robertuito_231', 'robertuito_244',
    'robertuito_298', 'robertuito_311', 'robertuito_328', 'robertuito_373',
    'robertuito_551', 'robertuito_625', 'robertuito_650'
]
columns_to_keep = ['date'] + \
    ['text'] + \
    [col for col in df_tweets_sample.columns if col.startswith('score_')] + \
    ['retweetCount', 'replyCount', 'likeCount', 'quoteCount'] + \
    selected_embeddings

df_tweets = df_tweets[columns_to_keep]